<a href="https://colab.research.google.com/github/emely3h/Geospatial_ML/blob/main/scripts/get_stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 0. Prepare Colab, Define Constants

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#! ls
%cd drive/MyDrive/MachineLearning/
#! git clone https://github.com/emely3h/Geospatial_ML.git
%cd Geospatial_ML
! ls

/content/drive/.shortcut-targets-by-id/15HUD3sGdfvxy5Y_bjvuXgrzwxt7TzRfm/MachineLearning
/content/drive/.shortcut-targets-by-id/15HUD3sGdfvxy5Y_bjvuXgrzwxt7TzRfm/MachineLearning/Geospatial_ML
data_exploration  experiments	     models	   pyproject.toml    scripts
docs		  image_processing   poetry.lock   README.md	     sripts
evaluation	  metrics_bug.ipynb  prepare_data  requirements.txt


In [3]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import pickle
from keras.utils import Sequence
from data_exploration.mask_stats import Mask_Stats
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import jaccard_score, recall_score, precision_score, f1_score, accuracy_score

In [4]:
class EvaluationMetrics:
    def __init__(self, iou, recall, precision, f1, accuracy):

        self.iou_invalid = iou[0]
        self.iou_valid = iou[1]
        self.iou_land = iou[2]
        self.mean_iou = iou.sum()/3

        self.precision_invalid = precision[0]
        self.precision_valid = precision[1]
        self.precision_land = precision[2]
        self.mean_precision = precision.sum()/3

        self.recall_invalid = recall[0]
        self.recall_valid = recall[1]
        self.recall_land = recall[2]
        self.mean_recall = recall.sum()/3

        self.f1_invalid = f1[0]
        self.f1_valid = f1[1]
        self.f1_land = f1[2]
        self.mean_f1 = f1.sum()/3

        self.mean_accuracy = accuracy

    def print_metrics(self):
        print(f"mean iou: {self.mean_iou}")
        print(f"iou invalid: {self.iou_invalid}")
        print(f"iou valid: {self.iou_valid}")
        print(f"iou land: {self.iou_land}\n")

        print(f"mean precision: {self.mean_precision}")
        print(f"precision_invalid: {self.precision_invalid}")
        print(f"precision_valid: {self.precision_valid}")
        print(f"precision_land: {self.precision_land}\n")

        print(f"mean recall: {self.mean_recall}")
        print(f"recall_invalid: {self.recall_invalid}")
        print(f"recall_valid: {self.recall_valid}")
        print(f"recall_land: {self.recall_land}\n")

        print(f"mean f1: {self.mean_f1}")
        print(f"f1_invalid: {self.f1_invalid}")
        print(f"f1_valid: {self.f1_valid}")
        print(f"f1_land: {self.f1_land}\n")

        print(f"mean accuracy: {self.mean_accuracy}")

### Metric calculation

In [8]:
total_tiles = 11121
train_tiles = 6672
test_val_tiles = 2224
data_path = "../data_colab/256_256"
experiment = "experiment_8"
batch_size = 32
tile_size = 256
step_size = 25
file_name = "5_3_layers_64_128_256_512_1024"

In [9]:
splits = ["train", "val", "test"]

for dataset in splits:
  print(f'starting with the {dataset} set.')
  true = np.memmap(os.path.join(data_path, f"{dataset}_split_y.npy"), mode="r", shape=(test_val_tiles, 256, 256), dtype=np.uint8)
  true = np.copy(true)
  pred = np.memmap(f"../models/{experiment}/predictions/pred_{dataset}_{file_name}.npy", mode="r", shape=(test_val_tiles, 256, 256, 3), dtype=np.float32)
  pred = np.copy(pred)
  pred = np.argmax(pred, axis=-1).astype(np.uint8)

  pred = pred.flatten()
  true = true.flatten()

  print(true.shape)
  print(type(true[0]))
  print(np.max(true))
  print(np.min(true))

  print(pred.shape)
  print(type(pred[0]))
  print(np.max(pred))
  print(np.min(pred))

  print('start calculate iou...')
  iou = jaccard_score(true, pred, average=None)
  print('start calculate recall...')
  recall = recall_score(true, pred, average=None)
  print('start calculate precision...')
  precision = precision_score(true, pred, average=None)
  print('start calculate f1...')
  f1 = f1_score(true, pred, average=None)
  print('start calculate accuracy...')
  accuracy = accuracy_score(true, pred)

  metrics = EvaluationMetrics(iou, recall, precision, f1, accuracy)

  with open(f"../metrics/{experiment}/metrics_{dataset}_{file_name}.pkl", 'wb') as file:
    pickle.dump(metrics, file)
  print('saving complete')
  print()
  print()


starting with the train set.
(145752064,)
<class 'numpy.uint8'>
2
0
(145752064,)
<class 'numpy.uint8'>
2
0
start calculate iou...
start calculate recall...
start calculate precision...
start calculate f1...
start calculate accuracy...
saving complete


starting with the val set.
(145752064,)
<class 'numpy.uint8'>
2
0
(145752064,)
<class 'numpy.uint8'>
2
0
start calculate iou...
start calculate recall...
start calculate precision...
start calculate f1...
start calculate accuracy...
saving complete


starting with the test set.
(145752064,)
<class 'numpy.uint8'>
2
0
(145752064,)
<class 'numpy.uint8'>
2
0
start calculate iou...
start calculate recall...
start calculate precision...
start calculate f1...
start calculate accuracy...
saving complete


